# Burgers 128 Experiment on Google Colab

This notebook runs the Burgers' equation experiment with 128 spatial points on Google Colab.

**Dataset:** Burgers' equation (1D nonlinear PDE)

**Resolution:** 128 spatial points

**Model:** UNet (synthetic/data-driven approach)

## 1. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Install Dependencies

In [ ]:
# Install required packages
!pip install -q hydra-core omegaconf phiflow==2.5.3 torch torchvision matplotlib tensorboard

## 3. Setup Project Paths

**⚠️ UPDATE THIS PATH to match where you uploaded your project in Google Drive!**

In [ ]:
import os
import shutil
from pathlib import Path

# ⚠️ UPDATE THIS PATH!
GDRIVE_PROJECT_PATH = "/content/drive/MyDrive/HYCO-PhiFlow"

# Local fast storage paths
LOCAL_PROJECT = "/content/HYCO-PhiFlow"
LOCAL_CACHE = "/content/cache"
LOCAL_DATA = "/content/data"
EXPERIMENT_NAME = "burgers_128"

# Create local directories
os.makedirs(LOCAL_CACHE, exist_ok=True)
os.makedirs(LOCAL_DATA, exist_ok=True)

print(f"✓ Google Drive project: {GDRIVE_PROJECT_PATH}")
print(f"✓ Local project: {LOCAL_PROJECT}")
print(f"✓ Local cache: {LOCAL_CACHE}/{EXPERIMENT_NAME}")
print(f"✓ Local data: {LOCAL_DATA}/{EXPERIMENT_NAME}")
print(f"✓ Experiment: {EXPERIMENT_NAME}")

## 4. Copy Code to Local Storage

In [ ]:
# Copy source code and configs to local storage (faster than Drive)
if os.path.exists(LOCAL_PROJECT):
    shutil.rmtree(LOCAL_PROJECT)

shutil.copytree(GDRIVE_PROJECT_PATH, LOCAL_PROJECT, 
                ignore=shutil.ignore_patterns('data', 'cache', 'outputs', 'results', 
                                               '__pycache__', '*.pyc', '.git', '*.ipynb'))

print("✓ Code copied to local storage")

# Change to project directory
%cd {LOCAL_PROJECT}

## 5. Check GPU Availability

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected! Go to Runtime → Change runtime type → Select GPU")

## 6. Generate Burgers Equation Data

Generate Burgers' equation simulation data at 128 spatial resolution.

**Note:** If you already have the data in Google Drive, skip to the next cell to copy it instead.

In [ ]:
# Generate Burgers equation data
!python run.py --config-name=burgers_quick_test \
    run_params.mode=[generate] \
    data.data_dir={LOCAL_DATA}/burgers_128 \
    data.cache_dir={LOCAL_CACHE}/burgers_128

## 6b. OR Copy Existing Data from Drive (Alternative)

If you already have generated data in Google Drive, copy it here instead of regenerating.

In [ ]:
# Uncomment and run this if you have existing data
# source_data = f"{GDRIVE_PROJECT_PATH}/data/burgers_128"
# if os.path.exists(source_data):
#     shutil.copytree(source_data, f"{LOCAL_DATA}/burgers_128")
#     print(f"✓ Copied burgers_128 data from Google Drive")
# else:
#     print(f"⚠️ Data not found at {source_data}")

## 7. Train UNet Model

Train the UNet model on Burgers' equation data.

**Training Configuration:**
- Batch size: 8
- Epochs: 1000 (adjust as needed)
- Prediction steps: 4
- Training simulations: 0-19
- Resolution: 128 points

In [ ]:
# Train the model with local cache for maximum speed
!python run.py --config-name=burgers_quick_test \
    run_params.mode=[train] \
    data.data_dir={LOCAL_DATA}/burgers_128 \
    data.cache_dir={LOCAL_CACHE}/burgers_128 \
    trainer_params.epochs=1000 \
    trainer_params.batch_size=8

## 8. Quick Training Test (Optional)

Run a short test with just 5 epochs to verify everything works before starting a long training session.

In [ ]:
# Quick test with 5 epochs
!python run.py --config-name=burgers_quick_test \
    run_params.mode=[train] \
    data.data_dir={LOCAL_DATA}/burgers_128 \
    data.cache_dir={LOCAL_CACHE}/burgers_128 \
    trainer_params.epochs=5 \
    trainer_params.batch_size=8

## 9. Evaluate the Model

Evaluate the trained model on test simulations.

In [ ]:
# Evaluate the trained model
!python run.py --config-name=burgers_quick_test \
    run_params.mode=[evaluate] \
    data.data_dir={LOCAL_DATA}/burgers_128 \
    data.cache_dir={LOCAL_CACHE}/burgers_128

## 10. Visualize Results (Optional)

Plot some predictions from the trained model.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# List available evaluation results
!ls -lth results/evaluation/

## 11. Monitor GPU Usage

In [ ]:
# Check current GPU usage
!nvidia-smi

## 12. View Training Logs

In [ ]:
# List recent outputs
!ls -lth outputs/

# View latest log file (adjust path as needed)
# !cat outputs/YYYY-MM-DD/HH-MM-SS/run.log

## 13. Save Results to Google Drive

**Important:** Run this cell to save your trained models and results before the session ends!

In [ ]:
from datetime import datetime

# Create timestamped backup folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_dir = f"{GDRIVE_PROJECT_PATH}/colab_results_burgers128_{timestamp}"
os.makedirs(backup_dir, exist_ok=True)

print("Saving results to Google Drive...")

# Copy results (trained models)
if os.path.exists("results"):
    shutil.copytree("results", f"{backup_dir}/results")
    print("✓ Saved trained models")

# Copy outputs (logs and metrics)
if os.path.exists("outputs"):
    shutil.copytree("outputs", f"{backup_dir}/outputs")
    print("✓ Saved logs and metrics")

# Save a summary file
summary = f"""Burgers 128 Experiment Results
Timestamp: {timestamp}
Experiment: burgers_128
Resolution: 128 spatial points
Model: UNet
PDE: Burgers' equation (1D nonlinear convection-diffusion)

Files saved:
- results/: Trained model checkpoints
- outputs/: Training logs and metrics
"""

with open(f"{backup_dir}/README.txt", "w") as f:
    f.write(summary)

print(f"\n✓ All results saved to: {backup_dir}")
print(f"\nYou can find your results in Google Drive at:")
print(f"{backup_dir}")

## 14. Download Results Locally (Optional)

Download results directly to your computer.

In [ ]:
from google.colab import files

# Create a zip file of results
!zip -r burgers128_results.zip results/ outputs/

# Download the zip file
files.download('burgers128_results.zip')

## Troubleshooting

### Out of Memory Error
If you get a CUDA OOM error, reduce the batch size:
```python
trainer_params.batch_size=4  # or even 2
```

### Session Timeout
- Run cell #13 periodically to save progress
- Consider using Colab Pro for longer sessions

### Cache Not Found
The code automatically caches data on first access. Wait for "Caching X simulations upfront..." to complete.

### Import Errors
Re-run cell #2 to reinstall dependencies.

## Experiment Variations

### Train for fewer epochs (testing)
```bash
trainer_params.epochs=100
```

### Use fewer training simulations
```bash
trainer_params.train_sim=[0,1,2,3,4]
```

### Adjust prediction steps
```bash
trainer_params.num_predict_steps=2
```

### Enable/disable sliding window
```bash
trainer_params.use_sliding_window=false
```

### Run full experiment (not quick test)
```bash
# Use burgers_experiment instead of burgers_quick_test
!python run.py --config-name=burgers_experiment \
    data.data_dir={LOCAL_DATA}/burgers_128 \
    data.cache_dir={LOCAL_CACHE}/burgers_128
```